In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
MODEL = 'llama2'

In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings


In [8]:
if MODEL.startswith('gpt'):
    model = ChatOpenAI(api_key=openai_api_key, model =MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model = MODEL)
    embeddings = OllamaEmbeddings()


In [9]:
model.invoke('define weather')

'\nWeather refers to the state of the atmosphere at a particular place and time, including various phenomena such as temperature, humidity, cloudiness, wind speed, and precipitation. The weather can vary greatly depending on the location and time of year, and can have a significant impact on daily activities and human health.\n\nThere are several types of weather conditions that can occur, including:\n\n1. Sunny weather: This occurs when the sky is clear and there is no cloud cover.\n2. Cloudy weather: This occurs when there are clouds in the sky, but they do not produce precipitation.\n3. Precipitation: This occurs when water droplets or ice crystals fall from the sky, such as rain, snow, sleet, or hail.\n4. Fog: This occurs when there is a thick layer of cloud close to the ground, reducing visibility.\n5. Wind: This occurs when there are gusts of air moving through the atmosphere.\n6. Thunderstorm: This occurs when there is a severe storm with thunder and lightning.\n7. Hail: This oc

In [10]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser

ans = chain.invoke('Tell me a joke')

In [11]:
print(ans)

Sure! Here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope that brought a smile to your face! Do you want to hear another one?


In [12]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('team6_serverprocessors.pdf')

In [13]:
chunks = loader.load_and_split()

In [14]:
len(chunks)

18

In [15]:
from langchain.prompts import PromptTemplate
template = '''
Answer the question based on the context given below
If you dont know the answer simply return 'I don\'t know'
Context: {context}
Question: {question}
'''

prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template=template
)

In [16]:
prompt.format(context= 'Here is a context',question ='Here is a question')

"\nAnswer the question based on the context given below\nIf you dont know the answer simply return 'I don't know'\nContext: Here is a context\nQuestion: Here is a question\n"

In [17]:
chain = prompt | model | parser

In [18]:
print('This will help understand the input that a chain expects')
chain.input_schema.schema()

This will help understand the input that a chain expects


{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [19]:
chain.invoke(
    {
        'context':'everyone at school calls me Utsav',
        'question':'what is my name?' 
    }
)

'Sure! Based on the context provided, your name is Utsav.'

In [20]:
from langchain_community.vectorstores import DocArrayInMemorySearch


In [21]:
vector_store = DocArrayInMemorySearch.from_documents(chunks, embedding=embeddings, )

In [28]:
vs_retriever = vector_store.as_retriever()

In [29]:
vs_retriever.invoke('What is the title?')

[Document(page_content='B. Specifications  \nThe AMD EPYC 9000 series is known for its Exceptional \nPerformance . It utilizes a Zen 4 architecture  for enhanced \nperformance  while offering  up to 128 cores, setting a new \nstandard for processing power.  It also i incorporates innovative \n3D V -Cache Technology, boosting cache capacity and \nimproving performance.  Similarly, it also supports high -\nbandwidth DDR5 memory, ensuring efficient data access and \nthroughput.  \nIn the security aspect, the processor is protected by A MD \nInfinity Guard, providing comprehensive security features to \nsafeguard critical data and infrastructure.  \nAMD EPYC™ processors are recognized as the most energy -\nefficient x86 servers in the industry.  Balances exceptional \nperformance with reduced energy c onsumption, leading to cost \nsavings for data center operators.', metadata={'source': 'team6_serverprocessors.pdf', 'page': 3}),
 Document(page_content='easily, and create the virtualized se

In [30]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | vs_retriever, 
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question":"What is the title of the paper?"})

'The title of the paper is "A Comparative Study of Intel Xeon Processors for Server Computing".'

In [31]:
questions = [
    'what are the topics covered in the paper?',
    'how many references are make in the paper?',
    'which processor is the best out of all?',
    'what are the strengths of intel\'s server processor?'
]

In [33]:
for q in questions:
    print(f'Question: {q}')
    print(f"Answer: {chain.invoke({'question':q})}")

Question: what are the topics covered in the paper?
Answer: Based on the provided reference list, the following are some of the topics covered in the paper:

1. Introduction to ARM processors and their types
2. Overview of Cortex M series processors with a deep explanation of each processor in this family
3. Comparison between ARM and Xeon processors in terms of time-to-solution and energy-to-solution
4. Discussion of the performance characteristics of Arm Neoverse N1 CPU in 7nm FinFET for infrastructure applications
5. Examination of Arm Neoverse V2 platform and its leadership performance and power efficiency for next-generation cloud computing, ML, and HPC workloads
6. Introduction to Skylake -SP processor and its 28-core Xeon processor
7. Review of the performance evaluation of sparse matrix multiplication kernels on Intel Xeon Phi
8. Explanation of the instruction-level energy model of Intel's Xeon Phi processor.
Question: how many references are make in the paper?
Answer: Based on